In [1]:
import pandas as pd

In [2]:
df2 = pd.read_csv("C:\\Users\Gebruiker\Desktop\pycharm\\new_merged_features_IC50_g12c.csv")
df2

<>:1: SyntaxWarning: invalid escape sequence '\G'
<>:1: SyntaxWarning: invalid escape sequence '\G'
C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_23796\2309530694.py:1: SyntaxWarning: invalid escape sequence '\G'
  df2 = pd.read_csv("C:\\Users\Gebruiker\Desktop\pycharm\\new_merged_features_IC50_g12c.csv")


,ChEMBL ID,Smiles,FC,N,H,Cl,Br,I,P,F,...,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,IC50 (nM)
0,1432260,COc1nc2nc(C)nc(N[C@H](C)c3cccc(c3C)C(F)(F)F)c2...,0,280,29,0,0,0,1,3,...,0,0,0,0,0,0,0,0,0,1.97
1,1432311,C[C@@H](Nc1nc(Cl)nc2cnc(cc12)P1(=O)CCN(CC1)C(C...,0,272,24,1,0,0,1,3,...,0,0,0,0,0,0,0,0,0,2.38
2,1348122,C[C@@H](Nc1nc(C)nc2ncc(cc12)N1CCN(CC1=O)C(C)=O...,0,254,25,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,2.53
3,1348107,C[C@@H](Nc1nc(C)nc2ncc(cc12)C1=CCN(CC1)C(C)=O)...,0,270,30,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,2.70
4,1348085,C[C@@H](Nc1nc(C)nc2nc(c(cc12)N1CC2(CN(C2)C(C)=...,0,294,26,0,0,0,0,6,...,0,0,0,0,0,0,0,0,0,2.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1850,1386298,COc1cc2nc(C)nc(NC(C)c3ccc(s3)-c3ccc(F)cc3CN)c2...,0,238,25,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,92.1
1851,1438494,C[C@@H](Nc1nn(C)c(=O)c2cc(=O)n(cc12)[C@H]1C[C@...,0,230,20,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,92.8
1852,1432129,C[C@@H](Nc1nc(C)nc2cnc(cc12)P1(=O)CCN(Cc2ccccc...,0,282,29,0,0,0,1,3,...,0,0,0,0,0,0,0,0,0,94.0
1853,1386990,CC(Nc1nc(C)nc2ccc(cc12)N1CCCC1)c1ccc(s1)-c1ccc...,0,252,33,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,94.2


In [3]:
import numpy as np
import pandas as pd
def pIC50(input):
    pIC50 = []

    input["IC50 (nM)"] = pd.to_numeric(input["IC50 (nM)"],errors='coerce')

    for i in input["IC50 (nM)"]:
        molar = i*(10**-9) # Converts nM to M
        pIC50.append(-np.log10(molar))

    input['pIC50'] = pIC50
    x = input["pIC50"]

    return x

In [4]:
filename = "C:\\Users\Gebruiker\Desktop\pycharm\\new_merged_features_IC50_g12c.csv"
#df = pd.read_csv("fda_original.csv",sep=";")
df2 = pd.read_csv(filename)
df2.dropna(subset=['IC50 (nM)'], inplace=True)
df2['IC50 (nM)'] = pIC50(df2)
df2['IC50 (nM)'] = df2['pIC50']  # Replace 'SV' column with pIC50 values

# Drop 'pIC50' column (optional) as it's now redundant
df2.drop(columns=['pIC50'], inplace=True)

df2

<>:1: SyntaxWarning: invalid escape sequence '\G'
<>:1: SyntaxWarning: invalid escape sequence '\G'
C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_23796\3414453163.py:1: SyntaxWarning: invalid escape sequence '\G'
  filename = "C:\\Users\Gebruiker\Desktop\pycharm\\new_merged_features_IC50_g12c.csv"


,ChEMBL ID,Smiles,FC,N,H,Cl,Br,I,P,F,...,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,IC50 (nM)
0,1432260,COc1nc2nc(C)nc(N[C@H](C)c3cccc(c3C)C(F)(F)F)c2...,0,280,29,0,0,0,1,3,...,0,0,0,0,0,0,0,0,0,8.705534
1,1432311,C[C@@H](Nc1nc(Cl)nc2cnc(cc12)P1(=O)CCN(CC1)C(C...,0,272,24,1,0,0,1,3,...,0,0,0,0,0,0,0,0,0,8.623423
2,1348122,C[C@@H](Nc1nc(C)nc2ncc(cc12)N1CCN(CC1=O)C(C)=O...,0,254,25,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,8.596879
3,1348107,C[C@@H](Nc1nc(C)nc2ncc(cc12)C1=CCN(CC1)C(C)=O)...,0,270,30,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,8.568636
4,1348085,C[C@@H](Nc1nc(C)nc2nc(c(cc12)N1CC2(CN(C2)C(C)=...,0,294,26,0,0,0,0,6,...,0,0,0,0,0,0,0,0,0,8.552842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1850,1386298,COc1cc2nc(C)nc(NC(C)c3ccc(s3)-c3ccc(F)cc3CN)c2...,0,238,25,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,7.035740
1851,1438494,C[C@@H](Nc1nn(C)c(=O)c2cc(=O)n(cc12)[C@H]1C[C@...,0,230,20,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,7.032452
1852,1432129,C[C@@H](Nc1nc(C)nc2cnc(cc12)P1(=O)CCN(Cc2ccccc...,0,282,29,0,0,0,1,3,...,0,0,0,0,0,0,0,0,0,7.026872
1853,1386990,CC(Nc1nc(C)nc2ccc(cc12)N1CCCC1)c1ccc(s1)-c1ccc...,0,252,33,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,7.025949


In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Load training descriptors data
#df1 = df1.loc[:, ~df1.columns.str.contains('Unnamed')]
df2.dropna(subset=['IC50 (nM)'], inplace=True)
X = df2.drop(columns=['IC50 (nM)','Smiles', 'ChEMBL ID'])
y = df2['IC50 (nM)']

# Initialize model and pipeline
model = RandomForestRegressor(max_depth=20, min_samples_leaf=10)
pipe = Pipeline([('scaler', StandardScaler()), ('model', model)])

# Load new descriptors data
new_descriptors = pd.read_csv('C:\\Users\Gebruiker\Desktop\pycharm\FDA_Hyb_Features.csv')
new_descriptors.dropna(inplace=True)

# Extract ChEMBL IDs and SMILES
chembl_id_column = new_descriptors['ChEMBL ID']
smiles_column = new_descriptors['Smiles']
new_descriptors.drop(columns=['ChEMBL ID', 'Smiles'], inplace=True)

# Ensure new_descriptors has the same columns as X
missing_cols = set(X.columns) - set(new_descriptors.columns)  # Columns missing in new data
extra_cols = set(new_descriptors.columns) - set(X.columns)  # Extra columns in new data

# Add missing columns with zero values
for col in missing_cols:
    new_descriptors[col] = 0

# Drop extra columns
new_descriptors = new_descriptors[X.columns]  # Reorder and drop extras

# Now X and new_descriptors have identical columns

# Initialize a DataFrame to store results
loop_results_df = pd.DataFrame(columns=['ChEMBL ID', 'Smiles', 'Predicted Value'])

# Number of iterations
loop_n = 50

# Dictionary to store predicted values for each ChEMBL ID
predicted_values = {chembl_id: [] for chembl_id in chembl_id_column.unique()}

# Run the loop to fit the model and predict the values
for i in range(loop_n):
    pipe.fit(X, y)

    loop_predicted_values = pipe.predict(new_descriptors)

    loop_df = pd.DataFrame({
        'ChEMBL ID': chembl_id_column,
        'Smiles': smiles_column,
        'Predicted Value': loop_predicted_values
    })
    loop_df.sort_values(by='Predicted Value', ascending=False, inplace=True)

    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=FutureWarning)
        loop_results_df = pd.concat([loop_results_df, loop_df.head(10)])

    # Store the predicted values for each ChEMBL ID
    for chembl_id, predicted_value in zip(chembl_id_column, loop_predicted_values):
        if chembl_id in predicted_values:
            predicted_values[chembl_id].append(predicted_value)

# Calculate frequency of each ChEMBL ID in the top 10
value_counts = loop_results_df['ChEMBL ID'].value_counts(normalize=True) * 10
#print(value_counts)

# Get the top 13 ChEMBL IDs
top_10_chembl_ids = value_counts.head(10).index

<>:20: SyntaxWarning: invalid escape sequence '\G'
<>:20: SyntaxWarning: invalid escape sequence '\G'
C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_23796\3534410684.py:20: SyntaxWarning: invalid escape sequence '\G'
  new_descriptors = pd.read_csv('C:\\Users\Gebruiker\Desktop\pycharm\FDA_Hyb_Features.csv')


In [6]:
mean_values = {chembl_id: np.mean(predicted_values[chembl_id]) for chembl_id in top_10_chembl_ids}
std_dev_values = {chembl_id: np.std(predicted_values[chembl_id]) for chembl_id in top_10_chembl_ids}

summary_df = pd.DataFrame({
    'ChEMBL ID': top_10_chembl_ids,
    'Frequency': [value_counts[chembl_id] for chembl_id in top_10_chembl_ids],
    'Avg Predicted Value': [mean_values[chembl_id] for chembl_id in top_10_chembl_ids]
})

summary_df.head(10)
summary_df.to_csv("RF_molecules_Newfeatures_G12C.csv", index=False)